In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import cnn_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore")

In [2]:
path_to_zip = tf.keras.utils.get_file(
    'data.zip',
    origin='http://aimaksen.bslience.cn/cats_and_dogs_filtered.zip',
    extract=True,
)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_dataset = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_dataset.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (75, 75))

validation_generator =  test_dataset.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary',
                                                          target_size = (75, 75))



Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [3]:
# val_batches = tf.data.experimental.cardinality(validation_dataset)
# test_dataset = validation_dataset.take(val_batches // 5)
# validation_dataset = validation_dataset.skip(val_batches // 5)

In [4]:
inital_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [5]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [6]:
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

In [8]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = inital_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [9]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281  

In [ ]:
initial_epochs = 20

history = model.fit_generator(train_generator,
                    validation_data = validation_generator,
                    epochs=initial_epochs)

Epoch 1/20
100/100 [==============================] - 24s 221ms/step - loss: 0.8286 - accuracy: 0.4859 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 2/20
100/100 [==============================] - 11s 110ms/step - loss: 0.7329 - accuracy: 0.4999 - val_loss: 0.6916 - val_accuracy: 0.5000
Epoch 3/20
100/100 [==============================] - 11s 114ms/step - loss: 0.7661 - accuracy: 0.4894 - val_loss: 0.6901 - val_accuracy: 0.5000
Epoch 4/20
100/100 [==============================] - 11s 112ms/step - loss: 0.7291 - accuracy: 0.5148 - val_loss: 0.6888 - val_accuracy: 0.5000
Epoch 5/20
100/100 [==============================] - 11s 106ms/step - loss: 0.7444 - accuracy: 0.4804 - val_loss: 0.6885 - val_accuracy: 0.5000
Epoch 6/20
100/100 [==============================] - 11s 106ms/step - loss: 0.7362 - accuracy: 0.5237 - val_loss: 0.6893 - val_accuracy: 0.5000
Epoch 7/20
100/100 [==============================] - 11s 107ms/step - loss: 0.7450 - accuracy: 0.4881 - val_loss: 0.6878 - val_ac

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range=range(initial_epochs)
plt.figure(figsize=(8,8))
plt.subplot(2,1,1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc,label="Validation Accuracy")
plt.legend()
plt.title("Training and Validation Accuracy")
 
plt.subplot(2,1,2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss,label="Validation Loss")
plt.legend()
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print('test accuracy: {:.2f}'.format(test_accuracy))

In [ ]:
label_dict = {
    0: 'cat',
    1: 'dog'
}

test_image_batch, test_label_batch = test_dataset.as_numpy_iterator().next()
test_image_batch = test_image_batch.astype('uint8')

cnn_utils.plot_predictions(model, test_image_batch, test_label_batch, label_dict, 32, 5, 5)

In [ ]:
test_image_batch, test_label_batch = train_dataset.as_numpy_iterator().next()

img_idx = 0
random_batch = np.random.permutation(np.arange(0,len(test_image_batch)))[:BATCH_SIZE]
image_activation = test_image_batch[random_batch[img_idx]:random_batch[img_idx]+1]

cnn_utils.get_activations(base_model, image_activation[0])

In [ ]:

cnn_utils.display_activations(cnn_utils.get_activations(base_model, image_activation[0]))